In [16]:
#Please run this code in Google Collab and upload the file task_fill_blacks.csv into the system
!pip install pystan~=2.14
!pip install fbprophet

import pandas as pd
import numpy as np
import tensorflow as tf
import io

from fbprophet import Prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
df = pd.read_csv("task_fill_blanks.csv", sep=";", decimal=".")

In [18]:
df

,time,energy,nitrogen_Nm3h
0,2022-10-20 19:09:15+00:00,14657.1150,20.976364
1,2022-10-20 19:09:30+00:00,7826.5360,20.900000
2,2022-10-20 19:09:45+00:00,8878.1625,20.586154
3,2022-10-20 19:10:00+00:00,1874.1160,21.177500
4,2022-10-20 19:10:15+00:00,11829.6765,20.780000
...,...,...,...
32936,2022-12-03 08:54:45+00:00,0.0000,0.000000
32937,2022-12-03 08:55:00+00:00,0.0000,0.000000
32938,2022-12-03 08:55:15+00:00,0.0000,0.000000
32939,2022-12-03 08:55:30+00:00,10643.8085,17.619231


In [19]:
#Step 1: divide this dataset into multiple df, whose data is from the first non-0 line to the last 0 line
# use fbprophet to each of such df to fit non-0 data, and predict the -13.37 lines
#for each df chunk, there will be 10 -13.37 lines, and 20 zero lines
print(df.iloc[0]["nitrogen_Nm3h"])

20.97636363636364


In [28]:
df_chunks = []
df_mini = {"time":[], "energy": [], "nitrogen_Nm3h": []}
i = 0
while i < len(df):
  energy = df.iloc[i]["energy"]
  nitro = df.iloc[i]["nitrogen_Nm3h"]
  time = df.iloc[i]["time"]
  if (energy > 0.0 or nitro > 0.0):
    df_mini["time"].append(time)
    df_mini["energy"].append(energy)
    df_mini["nitrogen_Nm3h"].append(nitro)
    #print(i)
    i = i + 1
  elif (energy == -13.37 or nitro == -13.37):
    i = i + 30
    df_mini = pd.DataFrame.from_dict(df_mini)
    df_chunks.append(df_mini)
    df_mini = {"time":[], "energy": [], "nitrogen_Nm3h": []}
    #print(i)
    continue
  


In [30]:
df_chunks[1]

,time,energy,nitrogen_Nm3h
0,2022-10-21 04:08:00+00:00,64991.4380,56.552500
1,2022-10-21 04:08:15+00:00,64971.2270,69.205714
2,2022-10-21 04:08:30+00:00,70240.8985,66.149286
3,2022-10-21 04:08:45+00:00,64970.5080,64.755455
4,2022-10-21 04:09:00+00:00,62972.0760,63.764000
5,2022-10-21 04:09:15+00:00,60587.1250,63.030909
6,2022-10-21 04:09:30+00:00,65083.0450,61.922857
7,2022-10-21 04:09:45+00:00,56225.8670,61.767857
8,2022-10-21 04:10:00+00:00,58506.4765,61.490714
9,2022-10-21 04:10:15+00:00,56241.6560,61.067143


In [33]:
forecasts_energy = []
forecasts_nitro = []
i = 0
for df_chunk in df_chunks:
  df_chunk['Year'] = df_chunk['time'].apply(lambda x: str(x)[:4])
  df_chunk['Month'] = df_chunk['time'].apply(lambda x: str(x)[5:7])
  df_chunk['Day'] = df_chunk['time'].apply(lambda x: str(x)[8:10])
  df_chunk['Hour'] = df_chunk['time'].apply(lambda x: str(x)[11:13])
  df_chunk['Min'] = df_chunk['time'].apply(lambda x: str(x)[14:16])
  df_chunk['Sec'] = df_chunk['time'].apply(lambda x: str(x)[17:19])
  df_chunk['ds'] = pd.DatetimeIndex(df_chunk['Year']+'-'+df_chunk['Month']+'-'+df_chunk['Day']+' '+ df_chunk['Hour']+':'+ df_chunk['Min']+':'+ df_chunk['Sec'])

  df_chunk = df_chunk.drop(columns = ['time', 'Year', 'Month', 'Day', 'Hour', 'Min', 'Sec'])

  df_train_energy = df_chunk.drop(columns = ['nitrogen_Nm3h'])
  df_train_nitro = df_chunk.drop(columns = ['energy'])

  df_train_nitro = df_train_nitro.rename(columns = {'nitrogen_Nm3h':'y'})
  df_train_energy = df_train_energy.rename(columns = {'energy':'y'})

  m1 = Prophet(interval_width = 0.95, daily_seasonality= True)
  m2 = Prophet(interval_width = 0.95, daily_seasonality= True)
  model1 = m1.fit(df_train_energy)
  model2 = m2.fit(df_train_nitro)

  forecast1 = model1.predict(m1.make_future_dataframe(periods=135, freq = 'S'))
  forecast2 = model2.predict(m2.make_future_dataframe(periods=135, freq = 'S'))

  forecasts_energy.append(forecast1)
  forecasts_nitro.append(forecast2)
  print(i)
  i += 1



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 18.


0


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 18.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 18.


1


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 18.


...

558


INFO:fbprophet:n_changepoints greater than number of observations. Using 18.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 18.


559


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 18.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 18.


560


In [34]:
forecasts_energy[0]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-10-20 19:09:15,-63.270675,2420.782586,17730.330030,-63.270675,-63.270675,10122.617769,10122.617769,10122.617769,10122.617769,10122.617769,10122.617769,0.0,0.0,0.0,10059.347094
1,2022-10-20 19:09:30,-57.404545,2159.730135,18411.444526,-57.404545,-57.404545,10115.202903,10115.202903,10115.202903,10115.202903,10115.202903,10115.202903,0.0,0.0,0.0,10057.798358
2,2022-10-20 19:09:45,-51.538416,2691.570316,17595.266694,-51.538416,-51.538416,10107.640980,10107.640980,10107.640980,10107.640980,10107.640980,10107.640980,0.0,0.0,0.0,10056.102564
3,2022-10-20 19:10:00,-45.672286,1866.913689,17300.332904,-45.672286,-45.672286,10099.932127,10099.932127,10099.932127,10099.932127,10099.932127,10099.932127,0.0,0.0,0.0,10054.259841
4,2022-10-20 19:10:15,-39.806156,1836.896312,17576.998370,-39.806156,-39.806156,10092.076476,10092.076476,10092.076476,10092.076476,10092.076476,10092.076476,0.0,0.0,0.0,10052.270320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2022-10-20 19:22:56,257.802163,1704.057595,17525.628178,257.802023,257.802294,9504.657726,9504.657726,9504.657726,9504.657726,9504.657726,9504.657726,0.0,0.0,0.0,9762.459889
178,2022-10-20 19:22:57,258.193238,1698.468928,17213.532003,258.193098,258.193371,9503.647799,9503.647799,9503.647799,9503.647799,9503.647799,9503.647799,0.0,0.0,0.0,9761.841037
179,2022-10-20 19:22:58,258.584314,1673.201359,17059.563741,258.584171,258.584448,9502.637265,9502.637265,9502.637265,9502.637265,9502.637265,9502.637265,0.0,0.0,0.0,9761.221579
180,2022-10-20 19:22:59,258.975389,1621.866059,17914.979415,258.975245,258.975525,9501.626124,9501.626124,9501.626124,9501.626124,9501.626124,9501.626124,0.0,0.0,0.0,9760.601513


In [51]:
energy_pred = []
nitro_pred = []
for energy in forecasts_energy:
  pred = energy.tail(135)
  #pred = pred.iloc[0:5:len(pred)-1]
  pred = pred["yhat"].tolist()
  pred = pred[0:len(pred):15]
  pred.append(pred[len(pred)-1])#the last -13.37 value I just copy the previous value
  energy_pred.append(pred)

In [52]:
energy_pred[0]

[9837.707657447338,
 9829.554648093126,
 9821.262992078957,
 9812.83292711863,
 9804.264693792627,
 9795.558533667903,
 9786.714691827263,
 9777.733415274366,
 9768.614953265762,
 9768.614953265762]

In [53]:
for nitro in forecasts_nitro:
  pred = nitro.tail(135)
  #pred = pred.iloc[0:5:len(pred)-1]
  pred = pred["yhat"].tolist()
  pred = pred[0:len(pred):15]
  pred.append(pred[len(pred)-1])#the last -13.37 value I just copy the previous value
  nitro_pred.append(pred)

In [54]:
nitro_pred[0]

[21.298202202310417,
 21.32428350455786,
 21.35103980702538,
 21.378471039066532,
 21.406577115932116,
 21.43535793794485,
 21.46481339131681,
 21.494943347737127,
 21.525747664096656,
 21.525747664096656]

In [58]:
energy_concat = []
for energy in energy_pred:
  for i in energy:
    energy_concat.append(i)

In [60]:
nitro_concat = []
for nitro in nitro_pred:
  for i in nitro:
    nitro_concat.append(i)

In [62]:
#create a copy of "fill in the black" df
df_copy = df.copy()
df_copy

,time,energy,nitrogen_Nm3h
0,2022-10-20 19:09:15+00:00,14657.1150,20.976364
1,2022-10-20 19:09:30+00:00,7826.5360,20.900000
2,2022-10-20 19:09:45+00:00,8878.1625,20.586154
3,2022-10-20 19:10:00+00:00,1874.1160,21.177500
4,2022-10-20 19:10:15+00:00,11829.6765,20.780000
...,...,...,...
32936,2022-12-03 08:54:45+00:00,0.0000,0.000000
32937,2022-12-03 08:55:00+00:00,0.0000,0.000000
32938,2022-12-03 08:55:15+00:00,0.0000,0.000000
32939,2022-12-03 08:55:30+00:00,10643.8085,17.619231


In [65]:
filled = 0 #indication of the number of rows being filled
for i in range(len(df_copy)):
  energy = df_copy.iloc[i]["energy"]
  nitro = df_copy.iloc[i]["nitrogen_Nm3h"]
  if energy == -13.37 and nitro == -13.37:
    df_copy.loc[i, ["energy"]] = [energy_concat[filled]]
    df_copy.loc[i,["nitrogen_Nm3h"]] = [nitro_concat[filled]]
    filled += 1

In [68]:
df_copy_2 = df_copy.copy()

In [70]:
# Using apply() method to combine two columns of text
df_copy_2["energy"] = df_copy_2["energy"].astype(str)
df_copy_2["time"] = df_copy_2["time"].astype(str)
df_copy_2["nitrogen_Nm3h"] = df_copy_2["nitrogen_Nm3h"].astype(str)
df_copy_2["time;energy;nitrogen_Nm3h"] = df_copy_2[["time", "energy","nitrogen_Nm3h"]].apply(";".join, axis=1)

In [71]:
df_copy_2

,time,energy,nitrogen_Nm3h,time;energy;nitrogen_Nm3h
0,2022-10-20 19:09:15+00:00,14657.115,20.97636363636364,2022-10-20 19:09:15+00:00;14657.115;20.9763636...
1,2022-10-20 19:09:30+00:00,7826.536,20.9,2022-10-20 19:09:30+00:00;7826.536;20.9
2,2022-10-20 19:09:45+00:00,8878.1625,20.586153846153845,2022-10-20 19:09:45+00:00;8878.1625;20.5861538...
3,2022-10-20 19:10:00+00:00,1874.116,21.1775,2022-10-20 19:10:00+00:00;1874.116;21.1775
4,2022-10-20 19:10:15+00:00,11829.6765,20.78,2022-10-20 19:10:15+00:00;11829.6765;20.78
...,...,...,...,...
32936,2022-12-03 08:54:45+00:00,0.0,0.0,2022-12-03 08:54:45+00:00;0.0;0.0
32937,2022-12-03 08:55:00+00:00,0.0,0.0,2022-12-03 08:55:00+00:00;0.0;0.0
32938,2022-12-03 08:55:15+00:00,0.0,0.0,2022-12-03 08:55:15+00:00;0.0;0.0
32939,2022-12-03 08:55:30+00:00,10643.8085,17.619230769230768,2022-12-03 08:55:30+00:00;10643.8085;17.619230...


In [72]:
df_copy_2 = df_copy_2.drop(columns = ["time","energy","nitrogen_Nm3h"])
df_copy_2

,time;energy;nitrogen_Nm3h
0,2022-10-20 19:09:15+00:00;14657.115;20.9763636...
1,2022-10-20 19:09:30+00:00;7826.536;20.9
2,2022-10-20 19:09:45+00:00;8878.1625;20.5861538...
3,2022-10-20 19:10:00+00:00;1874.116;21.1775
4,2022-10-20 19:10:15+00:00;11829.6765;20.78
...,...
32936,2022-12-03 08:54:45+00:00;0.0;0.0
32937,2022-12-03 08:55:00+00:00;0.0;0.0
32938,2022-12-03 08:55:15+00:00;0.0;0.0
32939,2022-12-03 08:55:30+00:00;10643.8085;17.619230...


In [73]:
df_copy_2.to_csv('furnace_filled.csv', index=False)